# pointwise でのランク学習

pointwise でのランク学習について、概要および実装例を調査します。

## (1) 概要

以下に、pointwise でのランク学習におけるインプット／アウトプットイメージを示します。

### (1-1)  他手法との比較



| 手法 | ランキング・スコア | ランキング・スコア計算（更新）方法 | クエリーID | 
| :---: | :---: | :--- | :--- | 
| pointwise | 絶対指標 | 各サンプルのfeature・適合度だけを用いる | 不要 | 
| pairwise | 相対指標 | ２件のサンプルのfeature・適合度のペアを使用 | 必要となる実装例が多い様子 | 
| listwise | 相対指標 | 各クエリーごとに、サンプルのfeature・適合度をすべて使用  | 必須 | 

### (1-2) 教師データ

素性（featureベクトルと等価）と、その適合度（＝ランク、おおむね１〜５の間の整数値）を教師データとして用意します。

適合度の値が大きいほど、ランキングが高い素性である、という意味になるようです。

- 教師データのフォーマット：(fit関数の引数。feature=10件とします)

| 適合度 | feature1 | feature2 | ・・・ | feature10 | 
| :---: | :---: | :---: | :---: | :---: | 
| 3 | 0.5 | 0.0 | ・・・ | 0.5 | 
| 2 | 0.2 | 1.0 | ・・・ | 0.1 | 
| 1 | 0.0 | 1.0 | ・・・ | 0.0 | 
| 3 | 0.0 | 0.1 | ・・・ | 0.2 | 
| 2 | 1.0 | 0.3 | ・・・ | 0.2 | 
| 1 | 1.0 | 0.5 | ・・・ | 1.0 | 


ただし、pointwise がそもそも、複数のサンプル間で比較を行う（＝pairwise/listwise）ようなアルゴリズムでないということから、クエリーIDなどの付加情報を必要としないようです。

このため、サンプルに付加された適合度は、学習セット全体における適合度ということになります。


### (1-3) 学習

教師データを学習処理に渡すと、モデルが生成されます。

モデルの内部には、featureごとの重みづけテーブルと、適合度ごとのスコア値テーブルを保持することが多いようです。

- 重み付けテーブルのイメージ

| feature1 | feature2 | ・・・ | feature10 | 
| :---: |  :---: | :---: |  :---: | 
| 0.1 | -0.05 | ・・・ | 0.3 | 


- 適合度ごとのスコア値テーブルのイメージ

| 適合度 | 1 | 2 | 3 | 4 | 5 |
| :---:| :---: |  :---: | :---: |  :---: | :---: | 
| スコア | 〜0.0 | 〜0.01 | 〜0.05 | 〜0.1 | 0.1〜 | 


### (1-4) 予測

テストデータを予測処理に渡すと、予測結果が生成され、テストデータについてのランキング・スコアが得られます。


- テストデータのフォーマット：(predict関数の引数＝前述の教師データのフォーマットと同一)

| ランク | feature1 | feature2 | ・・・ | feature10 | 
| :---: | :---: |  :---: | :---: |  :---: | 
| 1 | 0.0 |  0.0 | ・・・ |  0.0 | 
| 1 | 0.1 |  1.0 | ・・・ |  0.1 | 
| 1 | 0.5 |  0.0 | ・・・ |  0.3 | 

　　（テストデータの１番目の列の値は、予測処理には影響しないようです）


- 予測結果データのフォーマット：(予測処理の戻り＝テストデータと並びが同じになるようです)

| ランキング・スコア |
| :---: |
| 1 |
| 3 |
| 4 |

予測結果として得られるランキング・スコアは、教師データで用意された適合度のうちのいずれかの値（＝絶対指標）となるようです。

また、このランキング・スコアは他の手法と同様、テストデータをランキング・スコアの降順に並べ替えて、ランキングを得る・・・といった形で利用することになるかと存じます。

## (2) PRank/McRank の実装例

こちらを参考にしました。

http://d.hatena.ne.jp/kisa12012/20110805/1312535567

https://github.com/oiwah/ranking

PRank/McRank は pointwise ランク学習に分類されるようです。

### (2-1) GitHub からコードを取得

```
MacBookPro-makmorit-jp:~ makmorit$ cd GitHub
MacBookPro-makmorit-jp:GitHub makmorit$ git clone https://github.com/oiwah/ranking.git
Cloning into 'ranking'...
remote: Counting objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
MacBookPro-makmorit-jp:GitHub makmorit$
MacBookPro-makmorit-jp:GitHub makmorit$ ls -al
total 24
drwxr-xr-x   9 makmorit  staff    306 May 11 13:14 .
drwxr-xr-x+ 40 makmorit  staff   1360 May 11 13:14 ..
-rw-r--r--@  1 makmorit  staff  10244 May  9 12:35 .DS_Store
drwxr-xr-x   9 makmorit  staff    306 Feb  3 13:38 .git
drwxr-xr-x  34 makmorit  staff   1156 May 11 13:07 donusagi-bot
drwxr-xr-x  26 makmorit  staff    884 Feb  4 20:22 mecab
drwxr-xr-x  15 makmorit  staff    510 May  3 15:58 pyltr
drwxr-xr-x   8 makmorit  staff    272 May 11 13:14 ranking
drwxr-xr-x  17 makmorit  staff    578 May  3 10:31 rankpy
MacBookPro-makmorit-jp:GitHub makmorit$ 
```

### (2-2) ビルド実行

```
MacBookPro-makmorit-jp:GitHub makmorit$ cd ranking
MacBookPro-makmorit-jp:ranking makmorit$ ls -al
total 184
drwxr-xr-x   8 makmorit  staff    272 May 11 13:14 .
drwxr-xr-x   9 makmorit  staff    306 May 11 13:14 ..
drwxr-xr-x  13 makmorit  staff    442 May 11 13:17 .git
-rw-r--r--   1 makmorit  staff    482 May 11 13:14 README.md
drwxr-xr-x   6 makmorit  staff    204 May 11 13:14 src
drwxr-xr-x   4 makmorit  staff    136 May 11 13:14 test_data
-rwxr-xr-x   1 makmorit  staff  83005 May 11 13:14 waf
-rw-r--r--   1 makmorit  staff    463 May 11 13:14 wscript
MacBookPro-makmorit-jp:ranking makmorit$ 
MacBookPro-makmorit-jp:ranking makmorit$ waf configure build
-bash: waf: command not found
MacBookPro-makmorit-jp:ranking makmorit$ ./waf configure build
Setting top to                           : /Users/makmorit/GitHub/ranking 
Setting out to                           : /Users/makmorit/GitHub/ranking/build 
Checking for 'g++' (c++ compiler)        : ok 
'configure' finished successfully (0.139s)
Waf: Entering directory `/Users/makmorit/GitHub/ranking/build'
[1/5] cxx: src/pointwise/prank.cc -> build/src/pointwise/prank.cc.0.o
[2/5] cxx: src/pointwise/mcrank.cc -> build/src/pointwise/mcrank.cc.0.o
[3/5] cxx: src/test/test.cc -> build/src/test/test.cc.1.o
clangclang: : warningwarning: : argument unused during compilation: '-compatibility_version 1' [-Wunused-command-line-argument]argument unused during compilation: '-compatibility_version 1' [-Wunused-command-line-argument]

clang: warning: argument unused during compilation: '-current_version 1' [-Wunused-command-line-argument]
clang: warning: argument unused during compilation: '-current_version 1' [-Wunused-command-line-argument]
[4/5] cxxshlib: build/src/pointwise/prank.cc.0.o build/src/pointwise/mcrank.cc.0.o -> build/src/libranking.dylib
[5/5] cxxprogram: build/src/test/test.cc.1.o -> build/src/test/ranking_test
Waf: Leaving directory `/Users/makmorit/GitHub/ranking/build'
'build' finished successfully (0.857s)
MacBookPro-makmorit-jp:ranking makmorit$ 
```

### (2-3) テスト実行

提供されているテストデータ（学習セットおよび予測対象セット）はこちら

accuracy検証をやりやすくする意図かと思われますが、予測対象セットには、あらかじめ正解のランク（適合度）がつけられています。

（予測対象セットの適合度[＝レコードの１列目の値]は、予測処理においては意味を持ちません）

```
MacBookPro-makmorit-jp:src makmorit$ cat ~/GitHub/ranking/test_data/train.dat
4 1:1 2:4 3:6
4 1:2 2:3 3:5
3 1:3 2:4 3:4
3 1:6 2:1 3:5
2 1:5 2:4 3:3
2 1:4 2:3 3:3
1 1:8 2:2 3:1
1 1:7 2:5
0 1:10 2:2
0 1:12 2:3 3:1
MacBookPro-makmorit-jp:src makmorit$ cat ~/GitHub/ranking/test_data/test.dat
4 1:2 2:6 3:8
3 1:3 2:4 3:4
2 1:2 3:1
1 1:9 2:1 3:2
0 1:15
MacBookPro-makmorit-jp:src makmorit$
```

提供されているサンプル・プログラムを使って実行してみます。

５件の予測対象データのうち、４件が正解（＝すなわちランキングが当たっている）だった様子です。

```
MacBookPro-makmorit-jp:src makmorit$ test/ranking_test ~/GitHub/ranking/test_data/train.dat ~/GitHub/ranking/test_data/test.dat 
PRank
sample #0: score=4
sample #1: score=3
sample #2: score=2
sample #3: score=0
sample #4: score=0
accuracy : 4 / 5

McRank
sample #0: score=4
sample #1: score=3
sample #2: score=2
sample #3: score=0
sample #4: score=0
accuracy : 4 / 5

MacBookPro-makmorit-jp:src makmorit$ 
```